In [1]:
if 'google.colab' in str(get_ipython()):
    !pip uninstall tensorflow -y
    !pip install tensorflow==2.3.1 tensorflow-quantum
    !rm -rf quantum-gans
    !git clone https://github.com/WiktorJ/quantum-gans
    !cd quantum-gans; pip install .

In [153]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [154]:
import math
import random
import tensorflow as tf

import cirq
import numpy as np
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import  get_ground_state_for_g
from qsgenerator.states.simple_state_circuits import build_x_rotation_state
from qsgenerator.states.simple_rotation_generators import get_binary_x_rotation_provider, get_arcsin_x_rotation_provider
from qsgenerator.phase.analitical import construct_hamiltonian, get_theta_v, get_theta_w, get_theta_r, get_g_parameters_provider
from qsgenerator.evaluators.circuit_evaluator import CircuitEvaluator

In [155]:
generator_layers = 6
discriminator_layers = 7
data_bus_size = 5

In [156]:
real_phase = True
generic_generator = False
all_layers_labeling = False
full_layer_labeling = False
use_gen_label_qubit = False
use_disc_label_qubit = False
zxz = True

In [157]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(
    generator_layers, 
    discriminator_layers, 
    data_bus_size, 
    all_layers_labeling=all_layers_labeling,
    full_layer_labeling=full_layer_labeling,
    use_gen_label_qubit=use_gen_label_qubit,
    use_disc_label_qubit=use_disc_label_qubit)

In [158]:
if not generic_generator:
    gen, gs = build_ground_state_circuit(qubits=data_qubits, full_parametrization=True, zxz=True)

In [159]:
if real_phase:
    real, real_symbols = build_ground_state_circuit(qubits=data_qubits)
else:
    real, real_symbols = build_x_rotation_state(qubits=data_qubits)

In [160]:
pure_gen = gen.copy()
gen.append([disc])

In [161]:
pure_real = real.copy()
real.append([disc])

In [162]:
print("REAL GROUND STATE")
pure_real

REAL GROUND STATE


(0, 1): ───H───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │
(0, 2): ───────X───Z───Ry(theta_r)───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   │                                       │
(0, 3): ─────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───X─────────────@───X───────────────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                   │                                       │
(0, 4): ─────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───X─────────────@───X───────────────────────────────────@───X─────────────────────
                                                                                                                                                                                                                   │                                       │
(0, 5): ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───

In [163]:
print("GENERATOR")
pure_gen

GENERATOR


(0, 1): ───H───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │
(0, 2): ───────X───Z───Rz(theta0)───Rx(theta1)───Rz(theta2)───X────────────────────────────────────────@───X────────────────────────────────────────────────────────────────────────────────────────@───X────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                       │                                                                                            │
(0, 3): ──────────────────────────────────────────────────────Rz(theta9)───Rx(theta10)───Rz(theta11)───X───X───Rz(theta12)───Rx(theta13)───Rz(theta14)───X───Rz(theta3)───Rx(theta4)───Rz(theta5)───X───X───Rz(theta6)───Rx(theta7)───Rz(theta8)───X───X────────────────────────────────────────@───X────────────────────────────────────────────────────────────────────────────────────────@───X───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                                                                                                                                                                                                │                                                                                            │
(0, 4): ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Rz(theta9)───Rx(theta10)───Rz(theta11)───X───X───Rz(theta12)───Rx(theta13)───Rz(theta14)───X───Rz(theta3)───Rx(theta4)───Rz(theta5)───X───X───Rz(theta6)───Rx(theta7)───Rz(theta8)───X───X────────────────────────────────────────@───X────────────────────────────────────────────────────────────────────────────────────────@───X──────────────────────────────────────────────
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         │                                                                                            │
(0, 5): ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [122]:
print("DISCRIMINATOR")
disc

DISCRIMINATOR


(0, 0): ───Rx(d0)───Rz(d6)────ZZ────────────────Rx(d17)───Rz(d23)───ZZ────────────────Rx(d34)───Rz(d40)───ZZ────────────────Rx(d51)───Rz(d57)───ZZ────────────────Rx(d68)───Rz(d74)───ZZ────────────────Rx(d85)───Rz(d91)───ZZ─────────────────Rx(d102)───Rz(d108)───ZZ──────────────────
                              │                                     │                                     │                                     │                                     │                                     │                                        │
(0, 1): ───Rx(d1)───Rz(d7)────ZZ^d12───ZZ───────Rx(d18)───Rz(d24)───ZZ^d29───ZZ───────Rx(d35)───Rz(d41)───ZZ^d46───ZZ───────Rx(d52)───Rz(d58)───ZZ^d63───ZZ───────Rx(d69)───Rz(d75)───ZZ^d80───ZZ───────Rx(d86)───Rz(d92)───ZZ^d97───ZZ────────Rx(d103)───Rz(d109)───ZZ^d114───ZZ────────
                                       │                                     │                                     │                                     │                                     │                                     │                                         │
(0, 2): ───Rx(d2)───Rz(d8)────ZZ───────ZZ^d15───Rx(d19)───Rz(d25)───ZZ───────ZZ^d32───Rx(d36)───Rz(d42)───ZZ───────ZZ^d49───Rx(d53)───Rz(d59)───ZZ───────ZZ^d66───Rx(d70)───Rz(d76)───ZZ───────ZZ^d83───Rx(d87)───Rz(d93)───ZZ───────ZZ^d100───Rx(d104)───Rz(d110)───ZZ────────ZZ^d117───
                              │                                     │                                     │                                     │                                     │                                     │                                        │
(0, 3): ───Rx(d3)───Rz(d9)────ZZ^d13───ZZ───────Rx(d20)───Rz(d26)───ZZ^d30───ZZ───────Rx(d37)───Rz(d43)───ZZ^d47───ZZ───────Rx(d54)───Rz(d60)───ZZ^d64───ZZ───────Rx(d71)───Rz(d77)───ZZ^d81───ZZ───────Rx(d88)───Rz(d94)───ZZ^d98───ZZ────────Rx(d105)───Rz(d111)───ZZ^d115───ZZ────────
                                       │                                     │                                     │                                     │                                     │                                     │                                         │
(0, 4): ───Rx(d4)───Rz(d10)───ZZ───────ZZ^d16───Rx(d21)───Rz(d27)───ZZ───────ZZ^d33───Rx(d38)───Rz(d44)───ZZ───────ZZ^d50───Rx(d55)───Rz(d61)───ZZ───────ZZ^d67───Rx(d72)───Rz(d78)───ZZ───────ZZ^d84───Rx(d89)───Rz(d95)───ZZ───────ZZ^d101───Rx(d106)───Rz(d112)───ZZ────────ZZ^d118───
                              │                                     │                                     │                                     │                                     │                                     │                                        │
(0, 5): ───Rx(d5)───Rz(d11)───ZZ^d14────────────Rx(d22)───Rz(d28)───ZZ^d31────────────Rx(d39)───Rz(d45)───ZZ^d48────────────Rx(d56)───Rz(d62)───ZZ^d65────────────Rx(d73)───Rz(d79)───ZZ^d82────────────Rx(d90)───Rz(d96)───ZZ^d99─────────────Rx(d107)───Rz(d113)───ZZ^d116─────────────

In [123]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([0] * len(gs)) + \
                   np.random.normal(scale=eps, size=(len(gs),))
init_disc_weights = np.random.normal(size=(len(ds),))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [124]:
class CustomScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomScheduler, self).__init__()
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return max(math.e ** - ((step+200) / (self.warmup_steps / math.log(100))), 0.01)

In [125]:
learning_rate = CustomScheduler()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [126]:
#g_values = [0.1, 0.4, 0.25]
g_values = [-0.5]
# x_rotations = get_binary_x_rotation_provider({0: '100', 1: '011', 2: '101'})
# real_values_provider = get_arcsin_x_rotation_provider(g_values, data_bus_size)
real_values_provider = get_g_parameters_provider()

In [127]:
trainer = Trainer(g_values, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  real, 
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  real_values_provider = real_values_provider,
                  use_analytical_expectation=True)

In [ ]:
epochs = 5
disc_iteration = 20
gen_iteration = 5
snapshot_interval_epochs = 20
results = trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration,
      snapshot_interval_epochs=snapshot_interval_epochs)

In [128]:
def get_gen_for_g(g, gen_weights, gen_provider):
    rad = gen_provider(g)
    return np.append(gen_weights, rad)

def get_states_and_fidelty_for_real(gen_evaluator, 
                                    real_evaluator, 
                                    gen_weights, 
                                    g, 
                                    sized):
    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, gen_weights, gen_evaluator.label_value_provider))
    real = real_evaluator.get_state_from_params(real_evaluator.label_value_provider(g))
    return generated, real, cirq.fidelity(generated, real)

def get_states_and_fidelty_for_ground(gen_evaluator, g, gen_weights, size):

    generated = gen_evaluator.get_state_from_params(get_gen_for_g(g, gen_weights, gen_evaluator.label_value_provider), list(range(size)))
    ground = get_ground_state_for_g(g, size)
    return generated, ground, cirq.fidelity(generated, ground)

def compare_generated_for_g(gen_evaluator, g1, g2, gen_weights, size):
    generated1 = gen_evaluator.get_state_from_params(get_gen_for_g(g1, gen_weights, gen_evaluator.label_value_provider), list(range(size)))
    generated2 = gen_evaluator.get_state_from_params(get_gen_for_g(g2, gen_weights, gen_evaluator.label_value_provider), list(range(size)))
    return generated1, generated2, cirq.fidelity(generated1, generated2)

In [129]:
trained_disc_weights = tf.Variable(np.array([ 1.25888796e+01,  1.10409822e+01,  1.27487049e+01,  1.32927475e+01,
       -3.20522385e+01,  2.98508596e+00, -7.54223883e-01,  8.97036648e+00,
        8.98472309e+00, -2.77423954e+00,  8.90891266e+00,  5.72837019e+00,
        6.03105211e+00, -4.64482594e+00, -1.10843427e-01,  7.78598115e-02,
        3.00343895e+00,  3.83781940e-01,  6.08641243e+00,  8.64131927e+00,
       -2.17593918e+01,  1.45857897e+01,  2.36893883e+01,  8.75363445e+00,
       -2.42768993e+01,  1.27688437e+01,  2.53628349e+00,  1.39768391e+01,
        1.40961084e+01, -2.04474068e+01,  4.71392822e+00,  1.66926212e+01,
        1.88311214e+01,  9.17525005e+00,  1.16109962e+01,  1.08004580e+01,
        1.10795708e+01,  5.81477690e+00, -5.50215101e+00,  2.22007637e+01,
        2.25015125e+01,  3.21826210e+01, -1.80058708e+01, -7.85126591e+00,
       -7.77073908e+00,  1.40237570e+01,  3.14071465e+01, -1.07477732e+01,
        1.55852342e+00,  2.63786411e+01,  1.31890945e+01,  7.14759350e+00,
        5.46145630e+00, -1.79730053e+01,  7.40563774e+00,  1.06135674e+01,
       -1.55095673e+00, -2.66580944e+01,  6.51995277e+00, -2.37151980e+00,
        3.51896515e+01,  1.50027342e+01,  1.80733763e-02, -1.75052185e+01,
        9.60706902e+00, -6.06413984e+00,  5.35433817e+00, -7.88707399e+00,
        1.27363043e+01,  1.02446747e+01]), dtype=tf.float32)

trained_gen_weights01 = [
      0.05253782868385315,
      2.955197334289551,
      -6.483252048492432,
      1.9192016124725342,
      -4.1681294441223145,
      6.244518756866455,
      -1.2551707029342651,
      -2.4261536598205566,
      -11.947280883789062,
      -10.22300910949707,
      -1.3364331722259521,
      -8.297472953796387,
      -5.282688140869141,
      -0.13142277300357819,
      5.976173400878906,
      -3.357417106628418,
      -12.566207885742188,
      7.18119478225708,
      -16.498035430908203,
      3.4707891941070557,
      1.6033517122268677,
      0.8124145865440369,
      -13.593448638916016,
      -6.836384296417236,
      -2.0123815536499023,
      6.326815605163574,
      -1.846700668334961,
      -13.721556663513184,
      -3.0659067630767822,
      0.515267014503479,
      -7.4888596534729,
      -8.01744270324707,
      -5.027240753173828,
      3.0487253665924072,
      5.831473350524902,
      -0.8271010518074036,
      1.0818190574645996,
      7.898219585418701,
      2.8969829082489014,
      9.615506172180176,
      2.724125862121582,
      3.0620782375335693,
      -3.604370594024658,
      5.331075191497803,
      -1.321940541267395,
      -6.351419448852539,
      11.06411361694336,
      -7.704744815826416,
      7.086691379547119,
      1.5545109510421753,
      -9.210501670837402,
      -0.23868054151535034,
      7.672255516052246,
      2.568031072616577,
      -0.2664746344089508,
      -2.5799784660339355,
      5.718940734863281,
      -3.350531816482544,
      6.655004024505615,
      0.057653672993183136,
      -0.5741320848464966,
      -5.449305057525635,
      6.004470348358154,
      8.10859203338623,
      -9.0677490234375,
      3.4736056327819824,
      -5.204622745513916,
      2.480900526046753,
      -6.011523723602295,
      -10.655678749084473,
      -4.842355251312256,
      -1.866302251815796,
      -5.25300407409668,
      0.6689038276672363,
      -1.1704888343811035,
      6.318166255950928,
      10.36154556274414,
      6.099518299102783,
      -2.9995014667510986,
      1.4260350465774536,
      -8.489354133605957,
      -2.0282974243164062,
      0.4692971706390381,
      -1.559913158416748
    ]
trained_gen_weights04 = [
      -9.349910736083984,
      -3.137516498565674,
      0.11076110601425171,
      0.1079181432723999,
      -10.369482040405273,
      -1.4755661487579346,
      -7.087393283843994,
      5.457488536834717,
      -2.7765588760375977,
      3.089747667312622,
      12.335768699645996,
      -1.2149827480316162,
      3.585606813430786,
      -8.728550910949707,
      6.772098064422607,
      -2.2341201305389404,
      0.6231995224952698,
      1.1529603004455566,
      4.749046325683594,
      -0.30359596014022827,
      -6.343395233154297,
      3.299123764038086,
      0.01585269533097744,
      8.04002571105957
    ]
gen_symbols = gs 
# gen_symbols = gs
# real_symbols = ('r0', 'r1', 'r2')

In [130]:
gen_evaluator = CircuitEvaluator(pure_gen, gen_symbols, trainer.label_value_provider)
real_evaluator = CircuitEvaluator(pure_real, real_symbols, trainer.real_values_provider)

In [131]:
g = -0.5
get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, trained_gen_weights01, g, data_bus_size)

(array([-0.23467445-4.63969037e-02j, -0.2357525 +8.04925710e-02j,
        -0.1609043 -1.17724136e-01j, -0.03037973+1.15694806e-01j,
         0.16331296-3.34291235e-02j,  0.22748716-1.56355575e-01j,
        -0.14460447-4.84133326e-02j,  0.09213847+1.38567239e-02j,
         0.21165419+4.96481992e-02j,  0.21497248-5.16842529e-02j,
         0.1688731 +8.94004554e-02j,  0.01665199-9.94909033e-02j,
         0.06492839-6.71193423e-03j,  0.14277022-9.04331803e-02j,
        -0.20815049-3.12267579e-02j,  0.10334366+4.79909629e-02j,
         0.16212486+5.35890609e-02j,  0.1780254 -4.43736389e-02j,
         0.08346525+1.06237926e-01j,  0.0409586 -7.82387331e-02j,
        -0.14739616+2.40315199e-02j, -0.18653288+1.06191657e-01j,
         0.04068329+6.26537129e-02j, -0.04136726+1.23159029e-04j,
         0.21021084+1.25618890e-01j,  0.2561631 +2.40279846e-02j,
         0.14440167+1.29889309e-01j,  0.04938077-9.05234218e-02j,
         0.06787709+6.10272214e-02j,  0.19650622-5.48335910e-03j,
        -0

In [61]:
g = 0.1
get_states_and_fidelty_for_real(gen_evaluator, real_evaluator, trained_gen_weights01, g, data_bus_size)

(array([-0.6197622 -0.741683j  , -0.15156451+0.08932291j,
        -0.06467123+0.12837054j,  0.02567703-0.00468336j,
        -0.09434147+0.06165439j,  0.00830532+0.01887292j,
        -0.01917351+0.00240218j,  0.00198495+0.00198072j], dtype=complex64),
 array([ 0.92998064+0.j        ,  0.        -0.2924654j ,
         0.        -0.1898315j , -0.05969925+0.j        ,
         0.        -0.09346657j, -0.02939388+0.j        ,
        -0.01907879+0.j        ,  0.        +0.006j     ], dtype=complex64),
 0.972771800733657)

In [62]:
trained_gen_weights025 = [(abs(el1) + abs(el2))/2 for el1,el2 in zip(trained_gen_weights04,trained_gen_weights01)]

In [63]:
trained_gen_weights025

[9.353142738342285,
 3.0991290807724,
 4.836684435606003,
 0.24345017969608307,
 5.457694232463837,
 1.1369480192661285,
 5.8984055519104,
 3.5932573676109314,
 4.428778409957886,
 4.647344708442688,
 9.175678730010986,
 3.6802698373794556,
 3.741974949836731,
 5.504438757896423,
 3.803985893726349,
 5.01411235332489,
 5.079081565141678,
 2.2731891870498657,
 3.7765825986862183,
 1.0434789955615997,
 6.162328004837036,
 1.6687728017568588,
 1.8256146935746074,
 5.4393041133880615]

In [64]:
generated = gen_evaluator.get_state_from_params(get_gen_for_g(0.25, trained_gen_weights025, real_values_provider))
real = real_evaluator.get_state_from_params(real_values_provider(0.25))
generated, real, cirq.fidelity(generated, real)

(array([-0.13393262-0.2619366j ,  0.07937344-0.02939816j,
        -0.26901814-0.2608605j , -0.11164127+0.05950294j,
        -0.08629466+0.2538405j , -0.0899108 -0.05215018j,
         0.32074744+0.70841736j,  0.2456885 -0.04561926j], dtype=complex64),
 array([ 0.80998045+0.j        ,  0.        -0.40815178j,
         0.        -0.30263492j, -0.15249872+0.j        ,
         0.        -0.20913605j, -0.10538433+0.j        ,
        -0.07814   +0.j        ,  0.        +0.039375j  ], dtype=complex64),
 0.1260292243477048)

In [65]:
real025 = real_evaluator.get_state_from_params(real_values_provider(0.25))

In [66]:
real04 = real_evaluator.get_state_from_params(real_values_provider(0.4))
real01 = real_evaluator.get_state_from_params(real_values_provider(0.1))

In [68]:
cirq.fidelity(real025, -real025)

0.999999880790714

In [ ]:
compare_generated_for_g(gen_evaluator, -0.9, -0.8, trained_gen_weights, data_bus_size)

In [ ]:
g = 0
get_states_and_fidelty_for_ground(gen_evaluator, g, trained_gen_weights, data_bus_size)